This a UFC webscrapping and dataset manipulation code for all UFC fight data from 1993 until now. Gathered from http://ufcstats.com/statistics/events/completed


To begin we see an example of how the data looks raw on the website for the statistics. 

In [1]:
# looking at the advanced stats for a single fight 
import requests
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

fights = requests.get('http://www.ufcstats.com/fight-details/e91f289f1b9f9519').text
soupo = BeautifulSoup(fights, 'lxml')
data = []
for p in soupo.findAll('p', {"class":"b-fight-details__table-text"}):
    mystr=p.string
    mystr = re.sub(r'[\ \n]{2,}', '', str(mystr))
    if mystr != 'None':
        data.append(mystr)

#Printing just a few data values        
print(data[0:10])

['0', '1', '74 of 105', '89 of 209', '70%', '42%', '101 of 135', '109 of 229', '2 of 10', '0 of 0']


Next we examine the main loop for webscrapping the data, this is just an example from last weekends event. The goal here is to compile the stats located in various components within the website, each having its own HTML tag. Then eventually append the results into the data base, where this loop will automatically run weekly to update.

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# main loop for creating the advanced stats csv file

def get_winner(input_link):
    linko = []
    fight_winner = requests.get(input_link).text
    soup_winner = BeautifulSoup(fight_winner,'lxml')

    for everylink in soup_winner.findAll('div',{"class":"b-fight-details__person"}):
        linky = re.sub(r'[\ \n]{2,}', '', str(everylink.text))

        if linky[0:1] == 'W':
            if '"' in linky:
                num = linky.index('"')
                linko.append(linky[1:num])
            else:
                linko.append(linky[1:])

    #See if its a title fight

    if 'Title Bout' in soup_winner.find('i',{"class":"b-fight-details__fight-title"}).text:
        linko.append('Title Bout')

    else:
        linko.append('Non-Title Bout')

    for img in soup_winner.findAll('img'):		

        if 'http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/fight.png' == img.get('src'):
            linko.append("FIGHT")
        elif 'http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/perf.png' == img.get('src'):
            linko.append("PERF")
        elif 'http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/ko.png' == img.get('src'):
            linko.append("KO")
        elif 'http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/sub.png' == img.get('src'):
            linko.append("SUB")
    if len(linko) == 4:
        linko[2] = linko[2] + " " + linko[3]
        linko.pop()
    
    elif 'Title' in linko[0]:
        linko.insert(0,'Draw')

    elif len(linko) == 2:
        linko.append(" ")
    return(linko)



def getnames(input_link):
    z = 0
    names =[]
    fight_detail_request = requests.get(input_link).text
    soup_details = BeautifulSoup(fight_detail_request,'lxml')
    for link in soup_details.findAll('p', {"class":"b-fight-details__table-text"}):
        if z < 2:
            #link = re.sub(r'[\ \n]{2,}', '', str(link.text)
            name = link.text[1:len(link.text) -2]
            names.append(name)
            z +=1
        else:
            break

    for link in soup_details.title.findAll('h2',{"class":"b-content__title"}):
        names.append(link.text)

    wclass = soup_details.find('div',{"class":"b-fight-details__fight-head"}).text
    wclass = re.sub(r'[\ \n]{2,}', '', str(wclass))
    if 'Title' in wclass:
        weight = wclass[4:wclass.index("Title") -1]
        names.append(weight)
    else:
        names.append(wclass)
    return(names)

#print(getnames("http://www.ufcstats.com/fight-details/fae99a4089f8abb0"))
def getevents(input_link):
    fight_detail_request = requests.get(input_link).text
    soup_details = BeautifulSoup(fight_detail_request,'lxml')
    mylist = []
    myevent = soup_details.find('h2',{"class":"b-content__title"}).text
    myevent = re.sub(r'[\ \n]{2,}', '', str(myevent))
    mylist.append(myevent)
    for link in soup_details.findAll('li',{"class":"b-list__box-list-item"}):
        mystr = re.sub(r'[\ \n]{2,}', '', str(link.text))
        if 'Location' in mystr:
            mylist.append(mystr[9:])
        elif 'Date' in mystr:
            mylist.append(mystr[5:])
        else:
            mylist.append(mystr[11:])
    return(mylist)

#print(getevents("http://www.ufcstats.com/event-details/4956f60b7fa57c1a"))

def getstats(input_link):
    fight_stats = []

    fight_stats.extend(getref(input_link))
    fight_stats.extend(get_winner(input_link))
    fight_link = requests.get(input_link).text
    soup_fight = BeautifulSoup(fight_link, 'lxml')
    for stats in soup_fight.findAll('p', {"class":"b-fight-details__table-text"}):
            mystr=stats.string
            mystr = re.sub(r'[\ \n]{2,}', '', str(mystr))
            fight_stats.append(mystr)

    if fight_stats[1] == '1':
        fight_stats += 70 *['N/A']
        for i in range(51,133):
            fight_stats.insert(i,"N/A")

    elif fight_stats[1] == '2':
        fight_stats += 54 *['N/A']
        for i in range(71,131):
            fight_stats.insert(i,"N/A")

    elif fight_stats[1] == '3':
        fight_stats += 36 *['N/A']
        for i in range(91,131):
            fight_stats.insert(i,"N/A")

    elif fight_stats[1] == '4':
        fight_stats += 16 *['N/A']
        for i in range(110,131):
            fight_stats.insert(i,"N/A")


    return(fight_stats)



#132-53 for round 1 finises


def getref(input_link):
    #txt is the long string that is returned from soup
    txt = []
    #ref_clean is the clean appened list of things we want to use
    ref_clean = []
    fight_link = requests.get(input_link).text
    soup_ref = BeautifulSoup(fight_link, 'lxml')
    for refs in soup_ref.findAll('p', {"class":"b-fight-details__text"}):
        mystr = refs.text
        mystr = re.sub(r'[\ \n]{2,}', '', str(mystr))
        txt.append(mystr) 

    #Parce through the online long string that is fighter details

    Method_num = txt[0].index('Round')
    ref_clean.append(txt[0][7:Method_num])

    Round_num = txt[0].index('Time')
    ref_clean.append(txt[0][Method_num+6:Round_num])

    Rndnum = txt[0].index('Time format')
    ref_clean.append(txt[0][Round_num+5:Rndnum])

    Time_num = txt[0].index('Referee')
    ref_clean.append(txt[0][Rndnum+12:Time_num])

    referee = txt[0][Time_num+8:]
    ref_clean.append(referee)

    ref_clean.append(txt[1][8:])
    return(ref_clean)



#MAIN LOOP
def main():
    #main soup for events page
    source = requests.get('http://ufcstats.com/statistics/events/completed?page=all').text
    soup = BeautifulSoup(source, 'lxml')
    x=0


    with open(r'C:\Users\brock\OneDrive\Documents\quick_join.csv','w', newline='') as file:
        writer = csv.writer(file)
        csvheader = ['FIGHTER_RED','FIGHER_BLUE','WEIGHT','EVENT','DATE','STATE_COUNTRY','ATTENDENCE','FINISH','ROUND_FINISHED','TIME_FINISHED','SCHEDUELED ROUNDS','REFEREE','METHOD_OF_FINISH',
                     'WINNER','TITLE_OR_NOT','BONUS','?','?','KD_RED','KD_BLUE','SIG_STR_RED','SIG_STR_BLUE','SIG_STR_PERCENT_RED','SIG_STR_PERCENT_BLUE',
                     'TOTAL_STR_RED','TOTAL_STR_BLUE','TD_RED','TD_BLUE','TD_PERCENT_RED','TD_PERCENT_BLUE','SUB_ATT_RED','SUB_ATT_BLUE',
                     'REV_RED','REV_BLUE','CONTROL_RED','CONTROL_BLUE','?','?','RND1_KD_RED','RND1_KD_BLUE','RND1_SIG_STR_RED','RND1_SIG_STR_BLUE','RND1_SIG_STR_PERC_RED',
                     'RND1_SIG_STR_PERC_BLUE','RND1_TOTAL_STR_RED','RND1_TOTAL_STR_BLUE','RND1_TD_RED','RND1_TD_BLUE','RND1_TD_PERC_RED','RND1_TD_PERC_BLUE',
                     'RND1_SUB_ATT_RED','RND1_SUB_ATT_BLUE','RND1_REV_RED','RND1_REV_BLUE','RND1_CONTROL_RED','RND1_CONTROL_BLUE','?','?','RND2_KD_RED','RND2_KD_BLUE',
                     'RND2_SIG_STR_RED','RND2_SIG_STR_BLUE','RND2_SIG_STR_PERC_RED',
                     'RND2_SIG_STR_PERC_BLUE','RND2_TOTAL_STR_RED','RND2_TOTAL_STR_BLUE','RND2_TD_RED','RND2_TD_BLUE','RND2_TD_PERC_RED','RND2_TD_PERC_BLUE',
                     'RND2_SUB_ATT_RED','RND2_SUB_ATT_BLUE','RND2_REV_RED','RND2_REV_BLUE','RND2_CONTROL_RED','RND2_CONTROL_BLUE','?','?','RND3_KD_RED','RND3_KD_BLUE',
                     'RND3_SIG_STR_RED','RND3_SIG_STR_BLUE','RND3_SIG_STR_PERC_RED',
                     'RND3_SIG_STR_PERC_BLUE','RND3_TOTAL_STR_RED','RND3_TOTAL_STR_BLUE','RND3_TD_RED','RND3_TD_BLUE','RND3_TD_PERC_RED','RND3_TD_PERC_BLUE',
                     'RND3_SUB_ATT_RED','RND3_SUB_ATT_BLUE','RND3_REV_RED','RND3_REV_BLUE','RND3_CONTROL_RED','RND3_CONTROL_BLUE','?','?','RND4_KD_RED','RND4_KD_BLUE',
                     'RND4_SIG_STR_RED','RND4_SIG_STR_BLUE','RND4_SIG_STR_PERC_RED',
                     'RND4_SIG_STR_PERC_BLUE','RND4_TOTAL_STR_RED','RND4_TOTAL_STR_BLUE','RND4_TD_RED','RND4_TD_BLUE','RND4_TD_PERC_RED','RND4_TD_PERC_BLUE',
                     'RND4_SUB_ATT_RED','RND4_SUB_ATT_BLUE','RND4_REV_RED','RND4_REV_BLUE','RND4_CONTROL_RED','RND4_CONTROL_BLUE','?','?','RND5_KD_RED','RND5_KD_BLUE',
                     'RND5_SIG_STR_RED','RND5_SIG_STR_BLUE','RND5_SIG_STR_PERC_RED',
                     'RND5_SIG_STR_PERC_BLUE','RND5_TOTAL_STR_RED','RND5_TOTAL_STR_BLUE','RND5_TD_RED','RND5_TD_BLUE','RND5_TD_PERC_RED','RND5_TD_PERC_BLUE',
                     'RND5_SUB_ATT_RED','RND5_SUB_ATT_BLUE','RND5_REV_RED','RND5_REV_BLUE','RND5_CONTROL_RED','RND5_CONTROL_BLUE','?','?','SIG_STR_TOTAL_RED','SIG_STR_TOTAL_BLUE',
                     'SIG_STR_PERC_RED','SIG_STR_PERC_BLUE','HEAD_STR_RED','HEAD_STR_BLUE','BODY_STR_RED','BODY_STR_BLUE','LEG_STR_RED','LEG_STR_BLUE','DISTANCE_RED','DISTANCE_BLUE',
                     'CLINCH_RED','CLICH_BLUE','GROUD_RED','GROUND_BLUE','?','?','RND1_SIG_STR_RED','RND1_SIG_STR_BLUE','RND1_SIG_STR_PERC_RED','RND1_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
                     'RND1_HEAD_STR_BLUE','RND1_BODY_STR_RED','RND1_BODY_STR_BLUE','RND1_LEG_STR_RED','RND1_LEG_STR_BLUE','RND1_DISTANCE_RED','RND1_DISTANCE_BLUE',
                     'RND1_CLINCH_RED','RND1_CLINCH_BLUE','RND1_GROUND_RED','RND1_GOUND_BLUE','?','?','RND2_SIG_STR_RED','RND2_SIG_STR_BLUE','RND2_SIG_STR_PERC_RED','RND2_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
                     'RND2_HEAD_STR_BLUE','RND2_BODY_STR_RED','RND2_BODY_STR_BLUE','RND2_LEG_STR_RED','RND2_LEG_STR_BLUE','RND2_DISTANCE_RED','RND2_DISTANCE_BLUE',
                     'RND2_CLINCH_RED','RND2_CLINCH_BLUE','RND2_GROUND_RED','RND2_GOUND_BLUE','?','?','RND3_SIG_STR_RED','RND3_SIG_STR_BLUE','RND3_SIG_STR_PERC_RED','RND3_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
                     'RND3_HEAD_STR_BLUE','RND3_BODY_STR_RED','RND3_BODY_STR_BLUE','RND3_LEG_STR_RED','RND3_LEG_STR_BLUE','RND3_DISTANCE_RED','RND3_DISTANCE_BLUE',
                     'RND3_CLINCH_RED','RND3_CLINCH_BLUE','RND3_GROUND_RED','RND3_GOUND_BLUE','?','?','RND4_SIG_STR_RED','RND4_SIG_STR_BLUE','RND4_SIG_STR_PERC_RED','RND4_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
                     'RND4_HEAD_STR_BLUE','RND4_BODY_STR_RED','RND4_BODY_STR_BLUE','RND4_LEG_STR_RED','RND4_LEG_STR_BLUE','RND4_DISTANCE_RED','RND4_DISTANCE_BLUE',
                     'RND4_CLINCH_RED','RND4_CLINCH_BLUE','RND4_GROUND_RED','RND4_GOUND_BLUE','?','?','RND5_SIG_STR_RED','RND5_SIG_STR_BLUE','RND5_SIG_STR_PERC_RED','RND5_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
                     'RND5_HEAD_STR_BLUE','RND5_BODY_STR_RED','RND5_BODY_STR_BLUE','RND5_LEG_STR_RED','RND5_LEG_STR_BLUE','RND5_DISTANCE_RED','RND5_DISTANCE_BLUE',
                     'RND5_CLINCH_RED','RND5_CLINCH_BLUE','RND5_GROUND_RED','RND5_GOUND_BLUE']


        writer.writerow(csvheader)
        print('writing csv')


    for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
        print(x)
        #set x to a value to the number of desired events, new events can be concatenated later as events occur


        x = x + 1
        if x < 6:
            #check if its an event details link
            if "event-details" in link.get('href'):
                print(link.get('href'))
                #soup2 is the event details link
                #souping up the event link
                source2 = requests.get(link.get('href')).text
                soup2 = BeautifulSoup(source2, 'lxml')

                csvdata = getevents(link.get('href'))
                #looping through event link to get every fight-details link
                for links in soup2.findAll('a', attrs={'href': re.compile("^http://")}):

                    if "fight-details" in links.get('href'):
                        source3 = requests.get(links.get('href')).text
                        soup3 = BeautifulSoup(source3, 'lxml')
                        csvdata = getnames(links.get('href'))
                        csvdata.extend(getevents(link.get('href')))
                        csvdata.extend(getstats(links.get('href')))

                        print(getevents(link.get('href')))
                        
                        try:
                            if int(csvdata[7]) < 6 and int(csvdata[7]) != 0:
                                csvdata.insert(6,"0")
                        except:
                            continue
                            


                        with open(r'C:\Users\brock\OneDrive\Documents\quick_join.csv','a', newline='') as file:
                            writer = csv.writer(file)
                            #csv row is the stats list
                            writer.writerow(csvdata)
                            print('writing csv')

        else:
            break


main()

writing csv
0
1
2
3
http://ufcstats.com/event-details/e49c2db95e572dc8
4
http://ufcstats.com/event-details/d0d43cb9b14f231c
['UFC 256: Figueiredo vs. Moreno', 'December 12, 2020', 'Las Vegas, Nevada, USA']
writing csv
['UFC 256: Figueiredo vs. Moreno', 'December 12, 2020', 'Las Vegas, Nevada, USA']
writing csv
['UFC 256: Figueiredo vs. Moreno', 'December 12, 2020', 'Las Vegas, Nevada, USA']
writing csv
['UFC 256: Figueiredo vs. Moreno', 'December 12, 2020', 'Las Vegas, Nevada, USA']
writing csv
['UFC 256: Figueiredo vs. Moreno', 'December 12, 2020', 'Las Vegas, Nevada, USA']
writing csv
['UFC 256: Figueiredo vs. Moreno', 'December 12, 2020', 'Las Vegas, Nevada, USA']
writing csv
['UFC 256: Figueiredo vs. Moreno', 'December 12, 2020', 'Las Vegas, Nevada, USA']
writing csv
['UFC 256: Figueiredo vs. Moreno', 'December 12, 2020', 'Las Vegas, Nevada, USA']
writing csv
['UFC 256: Figueiredo vs. Moreno', 'December 12, 2020', 'Las Vegas, Nevada, USA']
writing csv
['UFC 256: Figueiredo vs. More

In [3]:
#importing CSV just created from previous look "quickjoin", also setting dataset display widths

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
advanced = pd.read_csv(r'C:\Users\brock\OneDrive\Documents\quick_join.csv')
#Dropping any possible duplicates
advanced.drop_duplicates(inplace = True)
advanced.head(3)

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,REV_RED,REV_BLUE,CONTROL_RED,CONTROL_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_REV_RED,RND1_REV_BLUE,RND1_CONTROL_RED,RND1_CONTROL_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_REV_RED,RND2_REV_BLUE,RND2_CONTROL_RED,RND2_CONTROL_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_REV_RED,RND3_REV_BLUE,RND3_CONTROL_RED,RND3_CONTROL_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_REV_RED,RND4_REV_BLUE,RND4_CONTROL_RED,RND4_CONTROL_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_REV_RED,RND5_REV_BLUE,RND5_CONTROL_RED,RND5_CONTROL_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
0,Deiveson Figueiredo,Brandon Moreno,Flyweight,UFC 256: Figueiredo vs. Moreno,"December 12, 2020","Las Vegas, Nevada, USA",0,Decision - Majority,5,5:00,5 Rnd (5-5-5-5-5),Jason Herzog,Derek Cleary46 - 48.Sal D'amato47 - 47.Junichi...,Draw,Title Bout,FIGHT,None,None,0

In [4]:
# All duplicated columns with the bonus as None have column formatting issue, columns are shifted to the right by 1.
try:
    p1 = advanced[(advanced.duplicated()) & (advanced['BONUS'] == 'None')].iloc[:,0:16]
    # Grabing the 2nd half of the data and shifting it over 1 column to the left to fix the formatting issue
    p2 = advanced[(advanced.duplicated()) & (advanced['BONUS'] == 'None')].iloc[:,16:].shift(1,axis='columns',fill_value = advanced[(advanced.duplicated()) & (advanced['BONUS'] == 'None')]['?.1'])
    df1 = pd.concat([p1,p2], sort = False, axis = 1)
    p3 = advanced[(advanced.duplicated()) & (advanced['BONUS'] != 'None')]
    df1 = pd.concat([df1,p3])

    # dropping all duplicated items then adding our corrected dataset 
    advanced.drop_duplicates(keep= False,inplace = True)
    advanced = pd.concat([advanced , df1])
    
except:
    print("No columns are out of place for this event due to double bonuses")
    df1 = pd.DataFrame()

No columns are out of place for this event due to double bonuses


In [5]:
# filling NAs as 0
advanced.fillna(value = 0,inplace = True)

In [6]:
advanced.head(3)

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,REV_RED,REV_BLUE,CONTROL_RED,CONTROL_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_REV_RED,RND1_REV_BLUE,RND1_CONTROL_RED,RND1_CONTROL_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_REV_RED,RND2_REV_BLUE,RND2_CONTROL_RED,RND2_CONTROL_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_REV_RED,RND3_REV_BLUE,RND3_CONTROL_RED,RND3_CONTROL_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_REV_RED,RND4_REV_BLUE,RND4_CONTROL_RED,RND4_CONTROL_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_REV_RED,RND5_REV_BLUE,RND5_CONTROL_RED,RND5_CONTROL_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
0,Deiveson Figueiredo,Brandon Moreno,Flyweight,UFC 256: Figueiredo vs. Moreno,"December 12, 2020","Las Vegas, Nevada, USA",0,Decision - Majority,5,5:00,5 Rnd (5-5-5-5-5),Jason Herzog,Derek Cleary46 - 48.Sal D'amato47 - 47.Junichi...,Draw,Title Bout,FIGHT,None,None,0

In [7]:
try:
    #Appears to be an issue with some columns being shifted from row 138 and onward (looking to fix this in the main loop in the future)
    s1 = advanced[advanced['HEAD_STR_RED'].str.contains('%',na=False)].iloc[:,0:137]
    s2 = advanced[advanced['HEAD_STR_RED'].str.contains('%',na=False)].iloc[:,137:].shift(-2,axis =1 )
    df2 = pd.concat([s1,s2], axis = 1)
    advanced = advanced[advanced['HEAD_STR_RED'].str.contains('%',na=False) == False]
    advanced = pd.concat([advanced,df2])
except:
    print("All Columns in place, no need for shift")
    

In [8]:
#shifted dataframe
advanced.tail(5)

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,REV_RED,REV_BLUE,CONTROL_RED,CONTROL_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_REV_RED,RND1_REV_BLUE,RND1_CONTROL_RED,RND1_CONTROL_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_REV_RED,RND2_REV_BLUE,RND2_CONTROL_RED,RND2_CONTROL_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_REV_RED,RND3_REV_BLUE,RND3_CONTROL_RED,RND3_CONTROL_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_REV_RED,RND4_REV_BLUE,RND4_CONTROL_RED,RND4_CONTROL_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_REV_RED,RND5_REV_BLUE,RND5_CONTROL_RED,RND5_CONTROL_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
8,Gavin Tucker,Billy Quarantillo,Featherweight Bout,UFC 256: Figueiredo vs. Moreno,"December 12, 2020","Las Vegas, Nevada, USA",0,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Jerin Valel,Dave Hagen27 - 30.Chris Lee27 - 30.Ron McCarth...,Gavin Tucker,Non-Title Bout,,Non

In [12]:
advanced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Columns: 244 entries, FIGHTER_RED to RND5_GOUND_BLUE
dtypes: float64(24), int64(14), object(206)
memory usage: 19.1+ KB


In [10]:
try:
    # similar column formation issue, creating corrected versions of the data
    t1 = advanced[advanced['GROUND_BLUE'] == 'None'].iloc[:,0:137]
    t2 = advanced[advanced['GROUND_BLUE'] == 'None'].iloc[:,137:].shift(1,axis=1)
    df3 = pd.concat([t1,t2],axis=1)
    advanced = advanced[advanced['GROUND_BLUE'] != 'None']
    advanced = pd.concat([advanced,df3])
    
except:
    print("Failed")



In [13]:
# removing unneeded columns
for i in advanced.columns:
    if i[0] == "?" or i[0:3] == 'Unn':
        print(i)
        advanced.drop(i,axis = 1,inplace=True)

?
?.1
?.2
?.3
?.4
?.5
?.6
?.7
?.8
?.9
?.10
?.11
?.12
?.13
?.14
?.15
?.16
?.17
?.18
?.19
?.20
?.21
?.22
?.23


In [14]:
#reseting the index 
advanced.reset_index(inplace = True)

In [15]:
#creating the loser stat, if the fighter isn't a winner and the fight wasn't a draw, that fighter was a loser
advanced['LOSER'] = 0
count = 0
for i in advanced[['FIGHTER_RED','FIGHER_BLUE','WINNER']].values:
    print(i)
    if i[2] == 'Draw':
        advanced['LOSER'].loc[count] = 'Draw'
    elif i[0] not in i[2]:
        advanced['LOSER'].loc[count] = i[0]
    elif i[1] not in i[2]:
        advanced['LOSER'].loc[count] = i[1]

    
        
    count += 1

['Deiveson Figueiredo' 'Brandon Moreno' 'Draw']
['Tony Ferguson' 'Charles Oliveira' 'Charles Oliveira']
['Mackenzie Dern' 'Virna Jandiroba' 'Mackenzie Dern']
['Junior Dos Santos' 'Ciryl Gane' 'Ciryl Gane']
['Cub Swanson' 'Daniel Pineda' 'Cub Swanson']
['Gavin Tucker' 'Billy Quarantillo' 'Gavin Tucker']
['Chase Hooper' 'Peter Barrett' 'Chase Hooper']
['Kevin Holland' 'Jacare Souza' 'Kevin Holland']
['Renato Moicano' 'Rafael Fiziev' 'Rafael Fiziev']
['Tecia Torres' 'Sam Hughes' 'Tecia Torres']


In [16]:
#fill NAs as 0
advanced.fillna(value = 0,inplace = True)

In [17]:
advanced.head(3)

,index,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,REV_RED,REV_BLUE,CONTROL_RED,CONTROL_BLUE,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_REV_RED,RND1_REV_BLUE,RND1_CONTROL_RED,RND1_CONTROL_BLUE,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_REV_RED,RND2_REV_BLUE,RND2_CONTROL_RED,RND2_CONTROL_BLUE,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_REV_RED,RND3_REV_BLUE,RND3_CONTROL_RED,RND3_CONTROL_BLUE,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_REV_RED,RND4_REV_BLUE,RND4_CONTROL_RED,RND4_CONTROL_BLUE,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_REV_RED,RND5_REV_BLUE,RND5_CONTROL_RED,RND5_CONTROL_BLUE,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE,LOSER
0,0,Deiveson Figueiredo,Brandon Moreno,Flyweight,UFC 256: Figueiredo vs. Moreno,"December 12, 2020","Las Vegas, Nevada, USA",0,Decision - Majority,5,5:00,5 Rnd (5-5-5-5-5),Jason Herzog,Derek Cleary46 - 48.Sal D'amato47 - 47.Junichi...,Draw,Title Bout,FIGHT,0,0,137 of 238,132 of 248,57%,53%,147 of 248,139 of 256,2 of 4,4 of 8,50%,50%,0,0,0,0,0:55,3:19,0,0,28 of

The final step is to append the data to the existing tables in the Postgre SQL AndNewAnalytics Database. 

In [18]:
import psycopg2
import requests
con = psycopg2.connect("Connection Details")
print("Database opened successfully")
print()
cur = con.cursor()
cur.execute('SELECT * from "Every_Fight"')
tupples = cur.fetchall()
headers = [i[0] for i in cur.description]
# Dataframe created from tupples and headers
Every_Fight = pd.DataFrame(tupples, columns=headers)

Every_Fight.tail(3) 

Database opened successfully



,Unnamed: 0,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,REV_RED,REV_BLUE,CONTROL_RED,CONTROL_BLUE,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_REV_RED,RND1_REV_BLUE,RND1_CONTROL_RED,RND1_CONTROL_BLUE,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_REV_RED,RND2_REV_BLUE,RND2_CONTROL_RED,RND2_CONTROL_BLUE,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_REV_RED,RND3_REV_BLUE,RND3_CONTROL_RED,RND3_CONTROL_BLUE,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_REV_RED,RND4_REV_BLUE,RND4_CONTROL_RED,RND4_CONTROL_BLUE,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_REV_RED,RND5_REV_BLUE,RND5_CONTROL_RED,RND5_CONTROL_BLUE,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE,LOSER
5899,2,Gabriel Benitez,Justin Jaynes,Lightweight Bout,UFC Fight Night: Hermansson vs. Vettori,"December 05, 2020","Las Vegas, Nevada, USA",0,KO/TKO,1,4:06,3 Rnd (5-5-5),0,Knee to Body At Distance,Gabriel Benitez,Non-Title Bout,PERF,1.0,0.0,38 of 104,22 of 52,36%,42%,38 of 104,22 of 52,0 of 0,0 of 0,---,---,0.0,0.0,0.0,0.0,0:00,0:00,1.0,0.0,38 of 104,22 of

In [19]:
#Renaming index to match database
advanced.rename({'index':'Unnamed: 0'}, inplace=True, axis = 1)

In [20]:
advanced.head(3)

,Unnamed: 0,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,REV_RED,REV_BLUE,CONTROL_RED,CONTROL_BLUE,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_REV_RED,RND1_REV_BLUE,RND1_CONTROL_RED,RND1_CONTROL_BLUE,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_REV_RED,RND2_REV_BLUE,RND2_CONTROL_RED,RND2_CONTROL_BLUE,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_REV_RED,RND3_REV_BLUE,RND3_CONTROL_RED,RND3_CONTROL_BLUE,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_REV_RED,RND4_REV_BLUE,RND4_CONTROL_RED,RND4_CONTROL_BLUE,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_REV_RED,RND5_REV_BLUE,RND5_CONTROL_RED,RND5_CONTROL_BLUE,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE,LOSER
0,0,Deiveson Figueiredo,Brandon Moreno,Flyweight,UFC 256: Figueiredo vs. Moreno,"December 12, 2020","Las Vegas, Nevada, USA",0,Decision - Majority,5,5:00,5 Rnd (5-5-5-5-5),Jason Herzog,Derek Cleary46 - 48.Sal D'amato47 - 47.Junichi...,Draw,Title Bout,FIGHT,0,0,137 of 238,132 of 248,57%,53%,147 of 248,139 of 256,2 of 4,4 of 8,50%,50%,0,0,0,0,0:55,3:19,0,0,

In [21]:
#Appending data to database, adding entries to the end.

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
engine = create_engine('Connection Details')
advanced.to_sql(
    'Every_Fight', 
    engine,
    index=False,
    if_exists = 'append',
    method='multi'
)

In [22]:
con = psycopg2.connect("Connection Details")
print("Database opened successfully")
print()
cur = con.cursor()
cur.execute('SELECT * from "Every_Fight"')
tupples = cur.fetchall()
headers = [i[0] for i in cur.description]
# Dataframe created from tupples and headers
Every_Fight = pd.DataFrame(tupples, columns=headers)


Every_Fight.tail(3) 

Database opened successfully



,Unnamed: 0,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,REV_RED,REV_BLUE,CONTROL_RED,CONTROL_BLUE,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_REV_RED,RND1_REV_BLUE,RND1_CONTROL_RED,RND1_CONTROL_BLUE,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_REV_RED,RND2_REV_BLUE,RND2_CONTROL_RED,RND2_CONTROL_BLUE,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_REV_RED,RND3_REV_BLUE,RND3_CONTROL_RED,RND3_CONTROL_BLUE,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_REV_RED,RND4_REV_BLUE,RND4_CONTROL_RED,RND4_CONTROL_BLUE,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_REV_RED,RND5_REV_BLUE,RND5_CONTROL_RED,RND5_CONTROL_BLUE,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE,LOSER
5909,4,Kevin Holland,Jacare Souza,Middleweight Bout,UFC 256: Figueiredo vs. Moreno,"December 12, 2020","Las Vegas, Nevada, USA",0,KO/TKO,1,1:45,3 Rnd (5-5-5),Mark Smith,Punch to Head From Bottom,Kevin Holland,Non-Title Bout,PERF,0.0,0.0,15 of 18,2 of 2,83%,100%,21 of 24,2 of 2,0 of 0,2 of 3,---,66%,0.0,0.0,0.0,0.0,0:02,1:27,0.0,0.0,15 of 18,2 of 2,83%,100

As we can see the new fights are successfully added to the existing fight database!

In [23]:
#writing to csv for extra protection
Every_Fight.to_csv(r'C:\Users\brock\OneDrive\Documents\UFC_Fight.csv')